# Topic Modeling of Climate Related News Articles

In this simple and short project I will use LDA to classify articles about *climate change* into different sub-topics.

In [1]:
# IMPORT NECESSARY LIBRARIES

import re # for text processing
import pandas as pd 

# gensim - for topic modeling
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# spacy - for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'said', 'says', 'say'])

from nltk.tokenize import word_tokenize

import itertools

import pickle

dir = "/path-to-my-dir"

# Import all articles
with open(dir+'all_articles.pkl', 'rb') as f:
    all_art = pickle.load(f)

/Users/magreco/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/magreco/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


## Data cleaning, text pre-processing
Let's clean the text a bit before we recover the topics.

In [2]:
#### Pre-process and tokenize text (no stemming)
# Define a function
def preprocess_text(text):

        # Remove all the special characters
        text = re.sub(r'\W', ' ', str(text))

        # remove all single characters
        text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

        # Substituting multiple spaces with single space
        text = re.sub(r'\s+', ' ', text, flags=re.I)

        # Converting to Lowercase
        text = text.lower()
        
        # To remove stop words
        stopW = stopwords.words('english')
    
        # Tokenize
        cleaned = word_tokenize(text)
        
        # Remove digits
        cleaned = [i for i in cleaned if not re.match(r'\d+', i)]
        
        # Remove Stopwords and single characters
        cleaned = [i for i in cleaned if i not in stopW and len(i) > 1]
        
        # Convert back to a string       
        preprocessed_text = ' '.join(cleaned)
        
        # Remove every word containing "_"
        preprocessed_text = re.sub(r'\s*(?:[\w_]*_(?:[\w_]*_)*[\w_]*)', '', preprocessed_text)
        
        # Remove every word containing numbers
        preprocessed_text = re.sub(r'\s*(?:[\w_]*[0-9+](?:[\w_]*[0-9+])*[\w_]*)', '', preprocessed_text)
        
        # Remove 2 characters words
        preprocessed_text = re.sub(r'\W*\b\w{1,2}\b', '', preprocessed_text)
        
        # Remove 3 characters words which are only made of consonants
        preprocessed_text = ' '.join(w for w in preprocessed_text.split() if 'a' in w or 'e' in w or 'i' in w or 'o' in w or 'u' in w and len(w)>3)
        
        # Remove consecutive identical words
        preprocessed_text = ' '.join(k for k, _ in itertools.groupby(preprocessed_text.split()))

        return preprocessed_text

In [3]:
# Create auxiliary data
climate_art = all_art
    
# Apply text pre-processing function to all articles
climate_art['trans_pp'] = climate_art['Transcript'].apply(lambda x:preprocess_text(x))

# Re-index the dataset
climate_art['Index'] = range(0, len(climate_art))

# Drop useless column
climate_art = climate_art.drop(columns = ['Unnamed: 0'])

# Assign each document an id
climate_art.set_index('Index', inplace = True)
climate_art['Document_No'] = range(0, len(climate_art))

# See what it looks like
climate_art.head(20)

,Source,Date,Program Name,Transcript,trans_pp,Document_No
Index,,,,,,
0,The Sunday Telegraph (LONDON),"February 01, 2004, Sunday",Business blasts Brown on the burden of tax col...,Wilcox's sentiments echo those of thousands of...,wilcox sentiments echo thousands businessmen t...,0
1,The Sunday Telegraph (LONDON),"February 08, 2004, Sunday",In search of another global calamity,APOCALYPSES in Maggie Gee's fiction are of the...,apocalypses maggie gee fiction peg variety vog...,1
2,The Sunday Telegraph (LONDON),"December 05, 2004, Sunday",All in the best possible taste (mostly) Whethe...,Nigella Lawson has made her name with the same...,nigella lawson made name kind gourmandist gust...,2
3,The Sunday Telegraph (LONDON),"December 12, 2004, Sunday",Flooding: don't let our defences down Resident...,Residents also dispute the accuracy of the Env...,residents also dispute accuracy environment ag...,3
4,The Sunday Telegraph (LONDON),"December 12, 2004, Sunday","Save the world, ignore global warming",Global warming will mainly harm the developing...,global warming mainly harm developing countrie...,4
5,The Sunday Telegraph (LONDON),"December 19, 2004, Sunday",Crime in a global climate Bah humbug! - There'...,"State of Fear (HarperCollins, pounds 17.99), M...",state fear harpercollins pounds michael cricht...,5
6,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",Landowners at war over wind farm threat to Bri...,Thousands of birds - including hundreds of gol...,thousands birds including hundreds golden eagl...,6
7,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",The Gulf Stream could fail . . . but I wouldn'...,Environmentalists are making much of the suppo...,environmentalists making much supposed threat ...,7
8,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",Plan your way around pitfalls,"If you're travelling on your own, ask about ro...",travelling ask room share schemes operators te...,8


### Lemmatization
Lemmatization is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form. For example, the lemma for *better* is *good*. The word *walk* is the base form for the word *walking*, and hence this is matched in both stemming and lemmatisation.

In [4]:
# LEMMATIZATION

transcript_pp = climate_art['trans_pp'].tolist()

data_words = [word_tokenize(i) for i in transcript_pp]

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Save objects as pickle and restart from here next time #
objects = [id2word, texts, corpus]

pickle.dump(objects, open(dir+'objects_topic_model.pkl', 'wb'))   

## Building the LDA Models
We now build different models defined by different number of topics. We then compute for each *coherence scores* so to choose eventually which model best suits our needs (ideally, one with high coherence). One good way to select the number of topics is to look at *coherence scores* with different number of topics and when these score stabilize (i.e. if you plot them, when the curve flattens), pick the one with highest coherence scores.

In [5]:
# Build LDA model for different number of topics 
lda_models = []
coherences = []
for i in range(8,26):
    print("Build model with ",i, " topics")
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    
    lda_models = lda_models + [lda_model]
    coherences = coherences + [coherence_lda]
    
# Store objects in a list and export them for future use
topic_analysis = [lda_models, coherences]
pickle.dump(topic_analysis, open(dir+'topic_analysis.pkl', 'wb'))

Build model with  8  topics

Coherence Score:  0.4290646377507996
Build model with  9  topics

Coherence Score:  0.42004449994374443
Build model with  10  topics

Coherence Score:  0.4501055801481438
Build model with  11  topics

Coherence Score:  0.45772409798498964
Build model with  12  topics

Coherence Score:  0.4399351772922007
Build model with  13  topics

Coherence Score:  0.4466017698366978
Build model with  14  topics

Coherence Score:  0.460766668985765
Build model with  15  topics

Coherence Score:  0.4828051202005461
Build model with  16  topics

Coherence Score:  0.47175372123279014
Build model with  17  topics

Coherence Score:  0.4880493037004091
Build model with  18  topics

Coherence Score:  0.46617008493121403
Build model with  19  topics

Coherence Score:  0.4704239455117802
Build model with  20  topics

Coherence Score:  0.49059174653821574
Build model with  21  topics

Coherence Score:  0.4784823505402219
Build model with  22  topics

Coherence Score:  0.4646131439

Let's now visualize the topics from the model we decide to pick (in this case, the one with 23 topics).

In [ ]:
# We pick 23 topics for this project
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_models[15], corpus, id2word)

In [7]:
pyLDAvis.display(vis)

We now want to use the chosen model to attach to the original dataset the following information:

- the topic number
- the probability of the article belonging to that topic
- the main keywords of the assigned topic

In [8]:
## Find dominant sentence/topic in each article
data_full = climate_art['Transcript'].tolist()

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_full):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: x[1], reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_models[15], corpus=corpus, texts=data_full)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Transcript2']

### Merge with original dataset
data_topics = pd.merge(climate_art, df_dominant_topic, on = 'Document_No', validate = '1:1')
data_topics.head(10)

,Source,Date,Program Name,Transcript,trans_pp,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Transcript2
0,The Sunday Telegraph (LONDON),"February 01, 2004, Sunday",Business blasts Brown on the burden of tax col...,Wilcox's sentiments echo those of thousands of...,wilcox sentiments echo thousands businessmen t...,0,6.0,0.2123,"tax, labour, year, rise, economy, price, pay, ...",Wilcox's sentiments echo those of thousands of...
1,The Sunday Telegraph (LONDON),"February 08, 2004, Sunday",In search of another global calamity,APOCALYPSES in Maggie Gee's fiction are of the...,apocalypses maggie gee fiction peg variety vog...,1,7.0,0.4143,"child, age, woman, also, family, include, year...",APOCALYPSES in Maggie Gee's fiction are of the...
2,The Sunday Telegraph (LONDON),"December 05, 2004, Sunday",All in the best possible taste (mostly) Whethe...,Nigella Lawson has made her name with the same...,nigella lawson made name kind gourmandist gust...,2,0.0,0.3062,"say, get, people, make, time, go, take, think,...",Nigella Lawson has made her name with the same...
3,The Sunday Telegraph (LONDON),"December 12, 2004, Sunday",Flooding: don't let our defences down Resident...,Residents also dispute the accuracy of the Env...,residents also dispute accuracy environment ag...,3,14.0,0.4409,"flood, water, say, people, area, home, storm, ...",Residents also dispute the accuracy of the Env...
4,The Sunday Telegraph (LONDON),"December 12, 2004, Sunday","Save the world, ignore global warming",Global warming will mainly harm the developing...,global warming mainly harm developing countrie...,4,22.0,0.2346,"would, may, even, many, claim, believe, make, ...",Global warming will mainly harm the developing...
5,The Sunday Telegraph (LONDON),"December 19, 2004, Sunday",Crime in a global climate Bah humbug! - There'...,"State of Fear (HarperCollins, pounds 17.99), M...",state fear harpercollins pounds michael cricht...,5,7.0,0.3872,"child, age, woman, also, family, include, year...","State of Fear (HarperCollins, pounds 17.99), M..."
6,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",Landowners at war over wind farm threat to Bri...,Thousands of birds - including hundreds of gol...,thousands birds including hundreds golden eagl...,6,17.0,0.1953,"say, government, would, change, climate, plan,...",Thousands of birds - including hundreds of gol...
7,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",The Gulf Stream could fail . . . but I wouldn'...,Environmentalists are making much of the suppo...,environmentalists making much supposed threat ...,7,0.0,0.3333,"say, get, people, make, time, go, take, think,...",Environmentalists are making much of the suppo...
8,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",Plan your way around pitfalls,"If you're travelling on your own, ask about ro...",travelling ask room share schemes operators te...,8,0.0,0.2503,"say, get, people, make, time, go, take, think,...","If you're travelling on your own, ask about ro..."
9,The Sunday Telegraph (LONDON),"March 07, 2004, Sunday",Clouds gather on future of US skiing Global wa...,"Like Europe, North America is currently enjoyi...",like europe north america currently enjoying s...,9,9.0,0.2637,"weather, year, day, summer, winter, snow, see,...","Like Europe, North America is currently enjoyi..."


# Et voilà, here is your dataset with each article assigned to a topic.